# 9.- Azure ML Deploy Training

In [41]:
import yaml

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml import Input, Output
from azure.ai.ml import command

In [42]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [43]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

environment_name = config["azure"]["environment_name"]
training_gpu_cluster = config["azure"]["training_gpu_cluster"]

data_asset_path = config["train"]["data_asset_path"]

In [44]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group_name,
    workspace_name
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [45]:
data_input = Input(
    path=config["train"]["data_asset_path"],
    type="uri_folder"
)

model_input = Input(
    path="../train/pretrained_models/yolo11n-seg.pt",
    type="uri_file"
)

In [ ]:


output = Output(
    type="uri_folder"
)

In [ ]:
command_job = """cp ${{ inputs.data }}/data.yaml /tmp/data.yaml && \
sed -i "s|path:.*$|path: ${{ inputs.data }}|" /tmp/data.yaml && \
python train.py"""

In [48]:
job = command(
    code="../src",
    command="python train.py",
    inputs={
        "data": data_input,
        "model": model_input,
    },
    outputs={
        "output_dir": output
    },
    environment_variables={ 
        "MODEL": model_input.path,
        "EPOCHS": "20",
        "BATCH": "4",
        "OUTPUT": output.path
    },
    environment=f"{environment_name}@latest",
    compute=training_gpu_cluster,
    experiment_name="yolo11-segmentation-training",
    display_name="yolo11-segmentation-training"
)

In [49]:
# Submit the job to Azure ML
job = ml_client.jobs.create_or_update(job)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
